# 导入数据

In [3]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [2]:
pip install pyLDAvis

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.4
    Uninstalling pandas-1.2.4:
      Successfully uninstalled pandas-1.2.4
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.


In [6]:
df = pd.read_csv('text_analysis_weibo.csv', index_col = 0)

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
df.head()

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
0,#高校通报教师图书馆打电话声音过大出言不逊#公道自在人心，谣言自在人心 ​​,0,0,0,a2331b38901d62d2d9a20529177ef3b3,0,22,湖北
1,转发C,0,0,0,d6dc4470f51fce93cc0cbad8abf55a75,0,33,广西
2,【#刘雨昕运动者联濛#河山覆冰雪，健儿迎冬奥[金牌]全能唱跳不设限，运动联濛开新年🇨🇳 期待...,0,0,0,372bc4782eb442b88035f920a7c1a68e,6,85,广东
3,丁程鑫//@丁程鑫后援会官博:#丁程鑫[超话]# ✨#丁程鑫 二十成金筑梦鑫世界# 大年初一...,0,0,0,6fe0d482bd3e78a3483e2a1d57f14ef2,75,1012,广东
4,诶，你们真不要脸诶。。。没资格宣传奥运。。。抵制抵制！,0,0,0,872380d71d6ee9130e8b49d331f2baa9,0,10,广东


# 语料预处理 

## 语料预处理 

In [8]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['标题/微博内容'][0]
remove_nums(test)[:100]

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'高校通报教师图书馆打电话声音过大出言不逊公道自在人心谣言自在人心'

## 分词

In [9]:
# 加载中文停用词词典，可个性化设置
# stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    # words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['标题/微博内容'][0]
clean_text(test)

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
Loading model cost 0.579 seconds.
Prefix dict has been built successfully.


'# 高校 通报 教师 图书馆 打电话 声音 过大 出言不逊 # 公道 自 在 人心 ， 谣言 自 在 人心   \u200b \u200b'

In [10]:
df['标题/微博内容'] = df['标题/微博内容'].astype(str)
df['微博内容分词'] = df['标题/微博内容'].apply(remove_nums)
df['微博内容分词'] = df['微博内容分词'].apply(clean_text)
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split())
df

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,#高校通报教师图书馆打电话声音过大出言不逊#公道自在人心，谣言自在人心 ​​,0,0,0,a2331b38901d62d2d9a20529177ef3b3,0,22,湖北,"[高校, 通报, 教师, 图书馆, 打电话, 声音, 过大, 出言不逊, 公道, 自, 在,..."
1,转发C,0,0,0,d6dc4470f51fce93cc0cbad8abf55a75,0,33,广西,[转发]
2,【#刘雨昕运动者联濛#河山覆冰雪，健儿迎冬奥[金牌]全能唱跳不设限，运动联濛开新年🇨🇳 期待...,0,0,0,372bc4782eb442b88035f920a7c1a68e,6,85,广东,"[刘雨昕, 运动, 者, 联, 濛, 河山, 覆, 冰雪, 健儿, 迎, 冬奥, 金牌, 全..."
3,丁程鑫//@丁程鑫后援会官博:#丁程鑫[超话]# ✨#丁程鑫 二十成金筑梦鑫世界# 大年初一...,0,0,0,6fe0d482bd3e78a3483e2a1d57f14ef2,75,1012,广东,"[丁程鑫, 丁程鑫, 后援会, 官博丁, 程鑫, 超话, 丁程鑫, 二十, 成金筑梦鑫, 世..."
4,诶，你们真不要脸诶。。。没资格宣传奥运。。。抵制抵制！,0,0,0,872380d71d6ee9130e8b49d331f2baa9,0,10,广东,"[诶, 你们, 真, 不要脸, 诶, 没, 资格, 宣传, 奥运, 抵制, 抵制]"
...,...,...,...,...,...,...,...,...,...
49995,935天气路况早知道 2022年3月2日 星期三 今日限行尾号3和8 8：00分发布 ...,0,0,0,2231ab79796d569625633acafd9f1351,445,74,河南,"[天气, 路况, 早, 知道, 年月日, 星期三, 今日, 限行, 尾号, 和, 分, 发布..."
49996,4月1日起，交通违法记分规则重大调整！ http://t.cn/A66zxfR0 ​​,0,0,0,91c9d380c1cc7811be5ee10ce510d77a,842,54,甘肃,"[月, 日, 起, 交通, 违法, 记分, 规则, 重大, 调整]"
49997,转发C,0,0,1,301771db13798e2dd493c02a44e59444,99,3433,江苏,[转发]
49998,全体 甘肃车主注意 4月1日起，全面实施！ ...,0,0,0,NaN,0,0,甘肃,"[全体, 甘肃, 车主, 注意, 月, 日, 起, 全面实施, 一次, 记分, 分值, 将,..."


In [ ]:
df

# LDA

In [11]:
dictionary = corpora.Dictionary(df['微博内容分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']]     # 根据分词结果创建语料库

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.024*"儿童" + 0.017*"孩子" + 0.017*"孤独症" + 0.013*"自闭症" + 0.010*"我"')
(1, '0.094*"的" + 0.025*"了" + 0.022*"是" + 0.020*"在" + 0.013*"也"')
(2, '0.029*"的" + 0.011*"月" + 0.010*"年" + 0.009*"市场" + 0.009*"在"')
(3, '0.029*"的" + 0.015*"和" + 0.013*"发展" + 0.010*"建设" + 0.009*"等"')
(4, '0.022*"的" + 0.010*"诈骗" + 0.009*"疫苗" + 0.008*"日" + 0.008*"月"')


C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
df['微博内容分词'].iloc[0]

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['高校',
 '通报',
 '教师',
 '图书馆',
 '打电话',
 '声音',
 '过大',
 '出言不逊',
 '公道',
 '自',
 '在',
 '人心',
 '谣言',
 '自',
 '在',
 '人心']

In [16]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8265488147735596	 
Topic: 0.024*"儿童" + 0.017*"孩子" + 0.017*"孤独症" + 0.013*"自闭症" + 0.010*"我" + 0.010*"孙颖莎" + 0.008*"世界" + 0.007*"康复" + 0.006*"岁" + 0.006*"刘雨昕"

Score: 0.1372811198234558	 
Topic: 0.094*"的" + 0.025*"了" + 0.022*"是" + 0.020*"在" + 0.013*"也" + 0.008*"有" + 0.007*"都" + 0.007*"和" + 0.007*"就" + 0.006*"这"

Score: 0.012129101902246475	 
Topic: 0.022*"的" + 0.010*"诈骗" + 0.009*"疫苗" + 0.008*"日" + 0.008*"月" + 0.007*"信息" + 0.007*"考生" + 0.007*"年" + 0.007*"考试" + 0.007*"在"

Score: 0.012070215307176113	 
Topic: 0.029*"的" + 0.011*"月" + 0.010*"年" + 0.009*"市场" + 0.009*"在" + 0.008*"和" + 0.007*"增长" + 0.007*"为" + 0.007*"汽车" + 0.007*"公司"

Score: 0.011970795691013336	 
Topic: 0.029*"的" + 0.015*"和" + 0.013*"发展" + 0.010*"建设" + 0.009*"等" + 0.007*"在" + 0.006*"新" + 0.006*"企业" + 0.005*"为" + 0.005*"项目"


C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
documents = df['微博内容分词'].values

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(0, 0.8265512), (1, 0.1372858), (2, 0.012069708), (3, 0.011970719), (4, 0.012122583)]

Document 2:
[(0, 0.100005575), (1, 0.10000555), (2, 0.5999778), (3, 0.10000555), (4, 0.10000557)]

Document 3:
[(0, 0.9406442), (2, 0.043488495)]

Document 4:
[(0, 0.9267971), (3, 0.050657973)]

Document 5:
[(0, 0.47658738), (1, 0.40283865), (2, 0.016747078), (3, 0.017114887), (4, 0.086711995)]

Document 6:
[(0, 0.100005575), (1, 0.10000555), (2, 0.5999778), (3, 0.10000555), (4, 0.10000557)]

Document 7:
[(0, 0.37687), (1, 0.47472513), (3, 0.13840458)]

Document 8:
[(0, 0.100005575), (1, 0.10000555), (2, 0.5999778), (3, 0.10000555), (4, 0.10000557)]

Document 9:
[(0, 0.7422889), (1, 0.16323338), (2, 0.07517911)]

Document 10:
[(0, 0.8353521), (1, 0.018491512), (2, 0.01868628), (3, 0.109035544), (4, 0.018434562)]



C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 可视化

In [19]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis)

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ImportError: cannot import name 'find_stack_level' from 'pandas.util._exceptions' (C:\Users\dell\anaconda3\lib\site-packages\pandas\util\_exceptions.py)

In [ ]:
# 导出可视化结果到html
# pyLDAvis.save_html(lda_vis, 'lda_visualization.html')

# 个人数据

In [5]:
df = pd.read_csv(r'搜狗全网新闻数据_utf-8_demo.csv',index_col=0,encoding='GBK')

NameError: name 'pd' is not defined

In [2]:
df.head()

NameError: name 'df' is not defined

In [3]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['content'][0]
remove_nums(test)[:100]

NameError: name 'df' is not defined

In [23]:
# 加载中文停用词词典，可个性化设置
# stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    # words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)

test = df['content'][0]
clean_text(test)

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'中广网 唐山 ６ 月 １ ２ 日 消息 （ 记者 汤一亮 \u3000 庄胜春 ） 据 中国 之声 《 新闻 晚 高峰 》 报道 ， 今天 （ １ ２ 日 ） 上午 ， 公安机关 ２ ０ １ ２ 年缉枪 制爆 专项 行动 “ 统一 销毁 非法 枪爆 物品 活动 ” 在 河北 唐山 正式 启动 ， １ ０ 万余只 非法 枪支 、 ２ ５ ０ 余吨 炸药 在 全国 １ ５ ０ 个 城市 被 统一 销毁 。 ； 泼 鳎 合衷 谖 倚布 ， 全国 缉枪 制爆 统一 销毁 行动 开始 ！ Ｋ 孀 殴安 部副 部长 黄明 一声令下 ， 大量 仿制 式 枪 以及 猎枪 、 火药枪 、 气枪 在 河北 唐山 钢铁厂 被 投入 炼钢炉 。 与此同时 ， 在 全国 各 省区市 １ ５ ０ 个 城市 ， 破案 追缴 和 群众 主动 上缴 的 １ ０ 万余支 非法 枪支 被 集中 销毁 ， 在 全国 各 指定 场所 ， ２ ５ ０ 余吨 炸药 被 分别 销毁 。 公安部 治安 局局长 刘 绍武 介绍 ， 这次 销毁 的 非法 枪支 来源于 三个 方面 。 Ａ 跎 芪 洌 捍 蚧 髌 瓢 赴括 涉黑 、 涉恶 的 团伙 犯罪 、 毒品 犯罪 ， 还有 从 境外 非法 走私 的 枪支 爆炸物 。 Ｔ 谙毁 现场 ， 记者 看到 了 被 追缴 和 上缴 的 各式各样 的 枪支 。 Ａ 跎 芪 洌 阂 舶括 制式 枪 ， 有的是 军用 枪 、 仿制 的 制式 抢 ， 还有 猎枪 、 私制 的 火药枪 等等 。 按照 我国 的 枪支 管理法 ， 这些 都 是 严厉 禁止 个人 非法 持有 的 。 中国 是 世界 上 持枪 犯罪 的 犯罪率 最低 的 国家 之一 。 Ｖ 忻懒手 破获 特大 跨国 走私 武器弹药 案 ＝ 日 ， 中 美 执法 部门 联手 成功 破获 特大 跨国 走私 武器弹药 案 ， 在 中国 抓获 犯罪 嫌疑人 ２ ３ 名 ， 缴获 各类 枪支 ９ ３ 支 、 子弹 ５ 万余 发及 大量 枪支 配件 。 在 美国 抓获 犯罪 嫌疑人 ３ 名 ， 缴获 各类 枪支 １ ２ 支 。 这是 公安部 与 美国移民 海关 执法局 通过 联合 调查 方式 侦破 重大 跨国 案件 的 又 一 成功 案例 。 ＃ 玻埃保 蹦 辏 冈 拢 玻 等 眨 上海浦东国际机场 海关 在 对 美国纽约 发 往

In [25]:
df['content'] = df['content'].astype(str)
df['微博内容分词'] = df['content'].apply(remove_nums)
df['微博内容分词'] = df['微博内容分词'].apply(clean_text)
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split())
df

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,contenttitle,content,微博内容分词
url,,,
http://news.sohu.com/20120612/n345428229.shtml,公安机关销毁１０余万非法枪支 跨国武器走私渐起,中广网唐山６月１２日消息（记者汤一亮 庄胜春）据中国之声《新闻晚高峰》报道，今天（１２日）上...,"[中广网, 唐山, 月, 日, 消息, 记者, 汤一亮, 庄胜春据, 中国, 之声, 新闻,..."
http://news.sohu.com/20120607/n344998325.shtml,张绍刚发道歉信网友不认可：他的问题是俯视他人（图）,天津卫视求职节目《非你莫属》“晕倒门”事件余波未了，主持人张绍刚前日通过《非你莫属》节目组发...,"[天津, 卫视, 求职, 节目, 非你莫属, 晕倒, 门, 事件, 余波, 未了, 主持人,..."
http://news.sohu.com/20120604/n344745879.shtml,＃（关注夏收）（３）夫妻“麦客”忙麦收,临沂（山东），２０１２年６月４日 夫妻“麦客”忙麦收 ６月４日，在山东省临沂市郯城县郯城街道...,"[临沂, 山东, 年月日, 夫妻, 麦客, 忙, 麦收, 月, 日, 在, 山东省, 临沂市..."
http://news.sohu.com/20120613/n345535702.shtml,欧洲杯大战在即 荷兰葡萄牙面临淘汰将背水一战,中广网北京６月１３日消息（记者王宇）据中国之声《新闻晚高峰》报道，明天凌晨两场欧洲杯的精彩比...,"[中广网, 北京, 月, 日, 消息, 记者, 王宇据, 中国, 之声, 新闻, 晚, 高峰..."
http://news.sohu.com/20120601/n344598651.shtml,扎克伯格携妻罗马当街吃３０元麦当劳午餐（组图）,环球网记者李亮报道，正在意大利度蜜月的“脸谱”创始人扎克伯格与他华裔妻子的一举一动都处于媒体...,"[环球网, 记者, 李亮, 报道, 正在, 意大利, 度蜜月, 的, 脸谱, 创始人, 扎克..."
...,...,...,...
http://news.cn.yahoo.com/ypen/20120531/1082314.html,联合国秘书长敦促叙利亚政府履行安南和平计划,中新网５月３１日电 据外电报道，联合国秘书长潘基文３１日敦促叙利亚政府尊重承诺，履行叙危机联...,"[中新网, 月, 日电, 据, 外电报道, 联合国, 秘书长, 潘基文, 日, 敦促, 叙利..."
http://news.cn.yahoo.com/ypen/20120531/1082438.html?f=E61_24_15,武汉一环卫工人因“占道”遭围殴 被泼滚烫米粉,荆楚网消息（记者张扬 通讯员殷莉红、刘永慧、刘妮）５月３１日，３名男子因嫌环卫车占道，竟将一...,"[荆楚网, 消息, 记者, 张扬, 通讯员, 殷莉红, 刘永慧, 刘妮, 月, 日名, 男子..."
http://news.cn.yahoo.com/newspic/news/23261/,女大学生夜行遇劫 遭割喉身亡（,nan,[]


In [26]:
dictionary = corpora.Dictionary(df['微博内容分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']]     # 根据分词结果创建语料库

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.052*"的" + 0.009*"和" + 0.009*"是" + 0.009*"在" + 0.007*"了"')
(1, '0.044*"的" + 0.009*"在" + 0.007*"和" + 0.007*"了" + 0.006*"投资"')
(2, '0.024*"的" + 0.012*"在" + 0.009*"了" + 0.008*"和" + 0.005*"枪支"')
(3, '0.030*"的" + 0.007*"在" + 0.006*"是" + 0.006*"高尔夫球场" + 0.005*"和"')
(4, '0.017*"的" + 0.005*"油价" + 0.005*"了" + 0.005*"在" + 0.004*"等"')


C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
df['微博内容分词'].iloc[0]

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['中广网',
 '唐山',
 '月',
 '日',
 '消息',
 '记者',
 '汤一亮',
 '庄胜春据',
 '中国',
 '之声',
 '新闻',
 '晚',
 '高峰',
 '报道',
 '今天',
 '日',
 '上午',
 '公安机关',
 '年缉枪',
 '制爆',
 '专项',
 '行动',
 '统一',
 '销毁',
 '非法',
 '枪爆',
 '物品',
 '活动',
 '在',
 '河北',
 '唐山',
 '正式',
 '启动',
 '万余只',
 '非法',
 '枪支',
 '余吨',
 '炸药',
 '在',
 '全国',
 '个',
 '城市',
 '被',
 '统一',
 '销毁',
 '泼',
 '鳎',
 '合衷',
 '谖',
 '倚布',
 '全国',
 '缉枪',
 '制爆',
 '统一',
 '销毁',
 '行动',
 '开始',
 '孀',
 '殴安',
 '部副',
 '部长',
 '黄明',
 '一声令下',
 '大量',
 '仿制',
 '式',
 '枪',
 '以及',
 '猎枪',
 '火药枪',
 '气枪',
 '在',
 '河北',
 '唐山',
 '钢铁厂',
 '被',
 '投入',
 '炼钢炉',
 '与此同时',
 '在',
 '全国',
 '各',
 '省区市',
 '个',
 '城市',
 '破案',
 '追缴',
 '和',
 '群众',
 '主动',
 '上缴',
 '的',
 '万余支',
 '非法',
 '枪支',
 '被',
 '集中',
 '销毁',
 '在',
 '全国',
 '各',
 '指定',
 '场所',
 '余吨',
 '炸药',
 '被',
 '分别',
 '销毁',
 '公安部',
 '治安',
 '局局长',
 '刘',
 '绍武',
 '介绍',
 '这次',
 '销毁',
 '的',
 '非法',
 '枪支',
 '来源于',
 '三个',
 '方面',
 '跎',
 '芪',
 '洌',
 '捍',
 '蚧',
 '髌',
 '瓢',
 '赴括',
 '涉黑',
 '涉恶',
 '的',
 '团伙',
 '犯罪',
 '毒品',
 '犯罪',
 '还有',
 '从',
 '境外',
 '非法',
 '走私',
 '的',
 '枪支',
 '爆炸物',
 

In [30]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.9984633326530457	 
Topic: 0.024*"的" + 0.012*"在" + 0.009*"了" + 0.008*"和" + 0.005*"枪支" + 0.005*"月" + 0.004*"将" + 0.004*"量化" + 0.004*"犯罪" + 0.004*"是"


C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
documents = df['微博内容分词'].values

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
# Function to infer topics for a document
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# Print topics for each document
for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(2, 0.99846333)]

Document 2:
[(0, 0.9978109)]

Document 3:
[(2, 0.9952811)]

Document 4:
[(0, 0.99818397)]

Document 5:
[(1, 0.9962489)]

Document 6:
[(0, 0.9988677)]

Document 7:
[(3, 0.9871064)]

Document 8:
[(4, 0.9973043)]

Document 9:
[(0, 0.99394757)]

Document 10:
[(2, 0.9888757)]



C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
pip list
conda remove d2l
conda remove pandas
conda install d2l
conda install pandas

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


SyntaxError: invalid syntax (<ipython-input-35-0b7f076fc385>, line 1)

In [37]:
conda update pandas

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Solving environment: ...working... 

Updating pandas is constricted by 

anaconda -> requires pandas==1.2.4=py38hd77b12b_0

If you are sure you want an update of your package either try `conda update --all` or install a specific version of the package you want using `conda install <pkg>=<version>`

done

## Package Plan ##

  environment location: C:\Users\dell\anaconda3

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-package-handling-2.2.0|   py38haa95532_1         292 KB
    conda-package-streaming-0.9.0|   py38haa95532_0          29 KB
    freetype-2.10.4            |       hd328e21_0         466 KB
    heapdict-1.0.1             |             py_0           9 KB
    jdcal-1.4.1                |             py_0          11 KB
    jsonschema-3.2.0           |             py_2          47 KB
    libzopfli-1.0.3            |       ha925a31_

In [39]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) # 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis)

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ImportError: cannot import name 'find_stack_level' from 'pandas.util._exceptions' (C:\Users\dell\anaconda3\lib\site-packages\pandas\util\_exceptions.py)

In [34]:
# 导出可视化结果到html
# pyLDAvis.save_html(lda_vis, 'lda_visualization.html')

C:\Users\dell\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
